# Model Selection 

The objective of this notebook is to build a very first ent to end Machine Learning model to predict the probability of a patient being discharged on a particular day.

We'll use the dataset generated by the `dataset` job of the ETL folder. 

At this instance, the focus won't be on model performance but rather on understanding the value potential of the available data and the speedness of the solution.

In [1]:
%cd /Users/josefinadallavia/Documents/MIM/Tesis/AML-hospital

/Users/josefinadallavia/Documents/MIM/Tesis/AML-hospital


In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
from matplotlib import pyplot as plt   
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from thesis_lib.utils import * 
from thesis_lib.modelling.data import *
from thesis_lib.modelling.model import *


/Users/josefinadallavia/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/josefinadallavia/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/josefinadallavia/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
data = Data().load('data/hospital_dataset')
variables = data.get_variables_dict()
data.get_stats()

Loading dataset:  hospital_train_data.parquet
Loading dataset:  hospital_val_data.parquet
Loading dataset:  hospital_test_data.parquet


dataset_type,train,val,test
n_observations,319150,33482,33309
relative_size,0.82694,0.0867542,0.0863059
n_cols,71,71,71
positives,42697,4555,4507
negatives,276453,28927,28802
positive_prop,0.133783,0.136043,0.135309
negative_prop,0.866217,0.863957,0.864691
min_date,2017-01-01,2018-11-11,2018-11-11
max_date,2018-11-10,2019-11-11,2019-11-11


In [4]:
sequence_features = ['labos_studies_names','sectors_names','images_study_types','surgeries_types',
                    'surgeries_injury_condition','surgeries_post_surgery_condition','surgeries_anesthesia_types']

In [5]:
text_features = ['administrative_diagnosis','presumptive_dianogsis','images_studies_names',
                'surgeries_actual']

In [6]:
text_features = [ 'administrative_diagnosis','presumptive_dianogsis','images_studies_names',
                'surgeries_actual' ,            
    'labos_studies_names','sectors_names','images_study_types','surgeries_types',
                    'surgeries_injury_condition','surgeries_post_surgery_condition','surgeries_anesthesia_types']

In [7]:
categorical_features = ['date_weekday',
 'request_origin',
 'origin',
 'entity_group',
 'gender',
 'request_sector',
 'insurance_entity',
 'admission_sector',
 'emergency_service',
 'isolation',
 'ARM_TEP',
 'CEC_TEP',
 'request_sector',
 'date_weekday',
 'admission_weekday',
 'date_month',
 'admission_month']

In [8]:
numerical_features = ['PIM2TEP',
 'emergency_service',
 'high_risk_TEP',
 'hosp_day_number',
 'images_count',
 'images_cumulative',
 'images_emergencies',
 'images_requester_roles_count',
 'images_requesters_count',
 'images_study_types_count',
 'labos_count',
 'labos_cumulative',
 'labos_emergencies',
 'labos_requester_roles_count',
 'labos_requesters_count',
 'labos_set_count',
 'labos_set_cumulative',
 'low_risk_TEP',
 'new_born_gestation_age',
 'new_born_weight',
 'patient_age',
 'sectors_count',
 'surgeries_count',
 'surgeries_cumulative',
 'surgeries_post_surgery_duration',
 'surgeries_pre_surgery_duration',
 'surgeries_prep_duration',
 'surgeries_services_count',
 'surgeries_surgery_delay',
 'surgeries_surgery_duration',
 'surgeries_types_count']

### All features

In [9]:
model_params = {'classifier': 'lgbm',
               'accepts_sparse': True,
                'categorical_features' : categorical_features ,
                'numerical_features' : numerical_features,
               'text_features': text_features,
                #'sequence_features':sequence_features
               }

In [10]:
lgbm_comb_features = Model(**model_params)
lgbm_comb_features.transform(data)

Fitting pipeline...
	 Preprocessing data
	 Preprocessing data
	 Encoding Categorical Features with OneHotEncoding
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
	 Preprocessing data
Transforming data...
	 Transforming numerical features
	 Preprocessing data
	 Transforming categorical features
	 Preprocessing data
	 Transforming text features
	 Preprocessing data
	 Transforming text features with TF-IDF embeddings
	 Transforming text features
	 Preprocessing data
	 Transforming text features with TF-IDF embeddings
	 Transforming text features
	 Preprocessing data
	 Transforming text features with TF-IDF embeddings
	 Transforming text features
	 Preprocessing data
	 Transforming text features with TF-IDF embeddings
	 Transforming text features
	 Preprocessing data
	 Transforming text features with TF-IDF embeddings
	 Transformi

In [11]:
lgbm_comb_features.n_features

20089

In [12]:
%%time
#lgbm_comb_features.fit_classifier()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 27.9 µs


In [13]:
#lgbm_comb_features.get_performance_metrics()

### Fine tuning

### Objective function

In [14]:
import lightgbm as lgb
import flatdict
from hyperopt import STATUS_OK

N_FOLDS = 3

# Create the dataset
train_set = lgb.Dataset(lgbm_comb_features.X_train,label=lgbm_comb_features.y_train)

def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    flatten_params = flatdict.FlatDict(params)
    parameters ={param_name: flatten_params[param_name] for param_name in flatten_params}
    
    
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    cv_results = lgb.cv(params = parameters, train_set = train_set,
                        nfold = n_folds,
                        early_stopping_rounds = 100, metrics = 'auc', seed = 2020,
                        verbose_eval=True)
  
    # Extract the best score
    best_score = max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

### Domain space

In [15]:
from hyperopt import hp

space_lgbm = {
    'objective': hp.choice('objective', ['binary']),
    'boosting_type': 'gbdt', 
    'subsample': hp.uniform('gdbt_subsample', 0.5, 1),
    'max_depth': hp.quniform('max_depth', 5, 30,1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
    'num_iterations': hp.choice('num_iterations', [100,150,200,300,500]) }

In [16]:
import hyperopt.pyll.stochastic 

### Optimization algorithm

In [17]:
from hyperopt import tpe
# Algorithm
tpe_algorithm = tpe.suggest

### Result history

In [18]:
from hyperopt import Trials
# Trials object to track progress
bayes_trials_lgbm = Trials()

### Optimization

In [ ]:
from hyperopt import fmin

MAX_EVALS = 200

# Optimize
best = fmin(fn = objective, space = space_lgbm, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials_lgbm)

  0%|          | 0/200 [00:00<?, ?trial/s, best loss=?]

/Users/josefinadallavia/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[1]	cv_agg's auc: 0.789407 + 0.00131796                
[2]	cv_agg's auc: 0.800836 + 0.00150509                
[3]	cv_agg's auc: 0.803995 + 0.00188585                
[4]	cv_agg's auc: 0.807389 + 0.00146032                
[5]	cv_agg's auc: 0.809759 + 0.0019068                 
[6]	cv_agg's auc: 0.81196 + 0.00211398                 
[7]	cv_agg's auc: 0.813462 + 0.00176015                
[8]	cv_agg's auc: 0.815378 + 0.00246355                
[9]	cv_agg's auc: 0.816973 + 0.00202205                
[10]	cv_agg's auc: 0.818241 + 0.00208835               
[11]	cv_agg's auc: 0.819413 + 0.00175557               
[12]	cv_agg's auc: 0.820406 + 0.00176739               
[13]	cv_agg's auc: 0.821358 + 0.00196775               
[14]	cv_agg's auc: 0.822212 + 0.00174893               
[15]	cv_agg's auc: 0.823157 + 0.00185084               
[16]	cv_agg's auc: 0.823931 + 0.00170187               
[17]	cv_agg's auc: 0.824645 + 0.00179197               
[18]	cv_agg's auc: 0.825644 + 0.00174771        

[147]	cv_agg's auc: 0.848064 + 0.00160153              
[148]	cv_agg's auc: 0.848096 + 0.00158397              
[149]	cv_agg's auc: 0.848141 + 0.00160573              
[150]	cv_agg's auc: 0.848187 + 0.00159885              
[151]	cv_agg's auc: 0.848208 + 0.00161232              
[152]	cv_agg's auc: 0.848243 + 0.0015676               
[153]	cv_agg's auc: 0.848262 + 0.00157023              
[154]	cv_agg's auc: 0.848293 + 0.00157529              
[155]	cv_agg's auc: 0.848319 + 0.00160576              
[156]	cv_agg's auc: 0.848324 + 0.00159402              
[157]	cv_agg's auc: 0.848378 + 0.00156582              
[158]	cv_agg's auc: 0.848432 + 0.00155226              
[159]	cv_agg's auc: 0.84846 + 0.00155211               
[160]	cv_agg's auc: 0.848471 + 0.00153101              
[161]	cv_agg's auc: 0.848501 + 0.00152342              
[162]	cv_agg's auc: 0.84851 + 0.00152346               
[163]	cv_agg's auc: 0.848538 + 0.00152555              
[164]	cv_agg's auc: 0.848541 + 0.00154082       

/Users/josefinadallavia/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



...

/Users/josefinadallavia/opt/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:503: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))



[1]	cv_agg's auc: 0.789407 + 0.00131796                                                  
[2]	cv_agg's auc: 0.791931 + 0.00142194                                                  
[3]	cv_agg's auc: 0.798823 + 0.000477491                                                 
[4]	cv_agg's auc: 0.801983 + 0.00130191                                                  
[5]	cv_agg's auc: 0.803538 + 0.00167191                                                  
[6]	cv_agg's auc: 0.804444 + 0.00146646                                                  
[7]	cv_agg's auc: 0.805022 + 0.00159497                                                  
[8]	cv_agg's auc: 0.806514 + 0.000375974                                                 
[9]	cv_agg's auc: 0.80755 + 0.0012965                                                    
[10]	cv_agg's auc: 0.808553 + 0.00155439                                                 
[11]	cv_agg's auc: 0.809739 + 0.0017212                                                  
[12]	cv_ag

[92]	cv_agg's auc: 0.832998 + 0.00196975                                                 
[93]	cv_agg's auc: 0.833131 + 0.00196545                                                 
[94]	cv_agg's auc: 0.833262 + 0.00194109                                                 
[95]	cv_agg's auc: 0.833377 + 0.00195709                                                 
[96]	cv_agg's auc: 0.833518 + 0.00194769                                                 
[97]	cv_agg's auc: 0.833647 + 0.00192869                                                 
[98]	cv_agg's auc: 0.833773 + 0.00190984                                                 
[99]	cv_agg's auc: 0.833916 + 0.00190736                                                 
[100]	cv_agg's auc: 0.834022 + 0.00191558                                                
[101]	cv_agg's auc: 0.83417 + 0.00189669                                                 
[102]	cv_agg's auc: 0.834284 + 0.00188525                                                
[103]	cv_a

[183]	cv_agg's auc: 0.840197 + 0.00162692                                                
[184]	cv_agg's auc: 0.840242 + 0.00163861                                                
[185]	cv_agg's auc: 0.840273 + 0.00163705                                                
[186]	cv_agg's auc: 0.840302 + 0.00162999                                                
[187]	cv_agg's auc: 0.840329 + 0.00163323                                                
[188]	cv_agg's auc: 0.84038 + 0.00163028                                                 
[189]	cv_agg's auc: 0.840436 + 0.00162815                                                
[190]	cv_agg's auc: 0.840498 + 0.00163515                                                
[191]	cv_agg's auc: 0.840537 + 0.00163677                                                
[192]	cv_agg's auc: 0.840564 + 0.00162893                                                
[193]	cv_agg's auc: 0.840611 + 0.00162733                                                
[194]	cv_a

[274]	cv_agg's auc: 0.842863 + 0.00168153                                                
[275]	cv_agg's auc: 0.842878 + 0.00168234                                                
 22%|██▎       | 45/200 [4:30:32<17:41:40, 410.97s/trial, best loss: 0.14964538354298662]

In [ ]:
!pip show hyperopt

In [ ]:
bayes_trials_lgbm.results